# setup

In [ ]:
import sys
sys.path.append('../../..')

In [ ]:
from copulae.input import generate_copula_net_input
from copulae.training.cflax.binorm import PositiveBiNormalCopula
from copulae.training.cflax.mixtures import NormalPDFNet

from copulae.training.loss import sq_error, sq_error_partial, copula_likelihood
from copulae.training import setup_training

In [ ]:
import jax.numpy as jnp
import jax.scipy.stats as jss
import jax

from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import optax
import pickle
from scipy.stats import bootstrap

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# data
https://github.com/yutingng/gen-AC.git

In [ ]:
import pandas as pd
import scipy
import copy

from sklearn.model_selection import train_test_split

In [ ]:
def add_train_random_noise(data, num_adds):
  new_data = np.random.rand(num_adds, data.shape[1])
  return np.concatenate((data, new_data), axis = 0)

In [ ]:
def rank_normalization(X):
  X = copy.deepcopy(X)
  for z in X:
      ndata = z.shape[0]
      gap = 1./(ndata+1)
      nfeats = z.shape[1]
      for i in range(nfeats):
          z[:, i] = scipy.stats.rankdata(z[:, i], 'ordinal')*gap
  return X

In [ ]:
!git clone https://github.com/yutingng/gen-AC.git

fatal: destination path 'gen-AC' already exists and is not an empty directory.


In [ ]:
class Boston():
  def __init__(self):
    # read
    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep = "\s+", skiprows = 22, header = None)
    X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    y = raw_df.values[1::2, 2]

    # split
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, random_state = 142857)
    X_train = np.concatenate((X_train, y_train[:, None]), axis = 1)
    X_test  = np.concatenate((X_test, y_test[:, None]), axis = 1)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # noise
    X_train = add_train_random_noise(X_train, int(X_train.shape[0]*0.01))

    # 2d
    train_data = X_train[:, [0, 13]]
    test_data = X_test[:, [0, 13]]

    # flip
    train_data[:, 0] = 1 - train_data[:, 0]
    test_data[:, 0] = 1 - test_data[:, 0]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [ ]:
class INTC_MSFT():
  def __init__(self):
    # read
    intel_f = open('gen-AC/data/raw/INTC_MSFT_GE/INTEL.data', 'r')
    intel = np.array(list(map(float, intel_f.readlines())))

    ms_f = open('gen-AC/data/raw/INTC_MSFT_GE/MS.data', 'r')
    ms = np.array(list(map(float, ms_f.readlines())))

    ge_f = open('gen-AC/data/raw/INTC_MSFT_GE/GE.data', 'r')
    ge = np.array(list(map(float, ge_f.readlines())))

    # split
    X = np.concatenate((intel[:, None], ms[:, None]), axis = 1)
    X_train, X_test, _, _ = train_test_split(X, X, shuffle = True, random_state = 142857)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # 2d, noise
    train_data = X_train[:, [0, 1]]
    train_data = add_train_random_noise(train_data, int(train_data.shape[0]*0.01))
    test_data = X_test[:, [0, 1]]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [ ]:
class GOOG_FB():
  def __init__(self):
    # read
    goog_f = open('gen-AC/data/raw/FB_GOOG/goog/close.vals', 'r')
    goog = np.array(list(map(float, goog_f.readlines())))

    fb_f = open('gen-AC/data/raw/FB_GOOG/fb/close.vals', 'r')
    fb = np.array(list(map(float, fb_f.readlines())))

    # split
    X = np.concatenate((goog[:, None], fb[:, None]), axis = 1)
    X_train, X_test, _, _ = train_test_split(X, X, shuffle=True, random_state=142857)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # 2d, noise
    train_data = X_train[:, [0, 1]]
    train_data = add_train_random_noise(train_data, int(train_data.shape[0]*0.01))
    test_data = X_test[:, [0, 1]]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [ ]:
def get_set(D_val, data_points):
  points = D_val
  points = jnp.expand_dims(points, axis=0)

  # PDF and CDF for X
  kde_x = jss.gaussian_kde(data_points[0], bw_method='silverman')
  density_x = kde_x.evaluate(points[0, 0, :])
  cumulative_x = jnp.array([kde_x.integrate_box_1d(-jnp.inf, p) for p in points[0, 0, :]])

  # PDF and CDF for Y
  kde_y = jss.gaussian_kde(D[1], bw_method='silverman')
  density_y = kde_y.evaluate(points[0, 1, :])
  cumulative_y = jnp.array([kde_y.integrate_box_1d(-jnp.inf, p) for p in points[0, 1, :]])

  I_pdf = density_x.T * density_y.T
  I_pdf = jnp.expand_dims(I_pdf, axis=0)
  cdf_xy = jnp.array((cumulative_x, cumulative_y))
  cdf_xy = jnp.expand_dims(cdf_xy, axis=0)

  del density_x
  del density_y
  del cumulative_x
  del cumulative_y

  return points, I_pdf, cdf_xy

In [ ]:
np.random.seed(30091985)
key = jax.random.PRNGKey(30091985)

# params

In [ ]:
layer_widths = [128, 64, 32, 16]
losses = [
    (0.01, sq_error),
    (0.5, sq_error_partial),
    (0.1, copula_likelihood),
]
lr = 2e-3
n_iter = 10000
label = '2catssg'

In [ ]:
def L_d(losses, params, state):
  loss = jnp.zeros((1,len(losses)), dtype=jnp.float32)
  for i, (w, loss_func) in enumerate(losses):
    loss = loss.at[0, i].set(w * loss_func(params, state))
  return loss

losses_eval = [
    (1.0, sq_error),
    (1.0, sq_error_partial),
    (1.0, copula_likelihood),
]

# boston

In [ ]:
model = PositiveBiNormalCopula(
    NormalPDFNet(layer_widths)
)

In [ ]:
data_loader = Boston()
D = np.array([data_loader.train_x, data_loader.train_y])[:, :, 0]
TrainingTensors = generate_copula_net_input(
    D=D,
    bootstrap=False
)

In [ ]:
nn_C, nn_dC, nn_c, cop_state, forward, grad = setup_training(
    model, TrainingTensors, losses
)

In [ ]:
_, subkey = jax.random.split(key)
init_params = model.init(subkey, TrainingTensors.UV_batches[0])
del subkey

params = init_params
optimizer = optax.adam(lr)
opt_state = optimizer.init(params)

In [ ]:
best = 1e6
for i in tqdm(range(n_iter)):
    grads, cop_state = grad(params, cop_state)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        loss = L_d(losses_eval, params, cop_state)
        if loss[0][-1] < best:
          best_params = params
          best_cop_state = cop_state
          best = loss[0][-1]
        print('Iter {}. Loss {}'.format(i, loss))

  0%|          | 1/10000 [01:13<203:11:52, 73.16s/it]

Iter 0. Loss [[ 0.05519268  0.26743177 18.215378  ]]


  0%|          | 12/10000 [02:12<3:15:03,  1.17s/it]

Iter 10. Loss [[0.05558956 0.25541455 6.8970304 ]]


  0%|          | 22/10000 [02:14<31:28,  5.28it/s]

...

100%|█████████▉| 9982/10000 [39:45<00:07,  2.43it/s]

Iter 9980. Loss [[0.26409972 0.03612038 0.45894942]]


100%|█████████▉| 9992/10000 [39:47<00:01,  5.53it/s]

Iter 9990. Loss [[0.26411438 0.03627694 0.45843267]]


100%|██████████| 10000/10000 [39:48<00:00,  4.19it/s]


In [ ]:
density_graph_points, I_pdf, cdf_xy = get_set(D, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))
np.mean(yhat)

-0.047525983

In [ ]:
D_val = np.array([data_loader.validation_x, data_loader.validation_y])[:, :, 0]
density_graph_points, I_pdf, cdf_xy = get_set(D_val, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))

L_d(losses_eval, best_params, best_cop_state), np.mean(yhat)

(Array([[ 0.29327896,  0.43385047, -1.1060357 ]], dtype=float32), -0.14973049)

In [ ]:
res = bootstrap(yhat, np.mean)
res.standard_error, res.confidence_interval

(0.1934671,
 ConfidenceInterval(low=-0.4581013480231057, high=0.3356706535112426))

In [ ]:
with open('drive/MyDrive/copulae/models/{}_boston.pkl'.format(label), 'wb') as fp:
    pickle.dump(best_params, fp)

# intc-msft

In [ ]:
model = PositiveBiNormalCopula(
    NormalPDFNet(layer_widths)
)

In [ ]:
data_loader = INTC_MSFT()
D = np.array([data_loader.train_x, data_loader.train_y])[:, :, 0]
TrainingTensors = generate_copula_net_input(
    D=D,
    bootstrap=False
)

In [ ]:
nn_C, nn_dC, nn_c, cop_state, forward, grad = setup_training(
    model, TrainingTensors, losses
)

In [ ]:
_, subkey = jax.random.split(key)
init_params = model.init(subkey, TrainingTensors.UV_batches[0])
del subkey

params = init_params
optimizer = optax.adam(lr)
opt_state = optimizer.init(params)

In [ ]:
best = 1e6
for i in tqdm(range(n_iter)):
    grads, cop_state = grad(params, cop_state)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        loss = L_d(losses_eval, params, cop_state)
        if loss[0][-1] < best:
          best_params = params
          best_cop_state = cop_state
          best = loss[0][-1]
        print('Iter {}. Loss {}'.format(i, loss))

  0%|          | 1/10000 [00:59<166:06:47, 59.81s/it]

Iter 0. Loss [[ 0.07117352  0.27198154 18.428902  ]]


  0%|          | 12/10000 [01:58<3:15:16,  1.17s/it]

Iter 10. Loss [[0.07158655 0.26004973 7.0295515 ]]


  0%|          | 21/10000 [02:00<57:37,  2.89it/s]

...

100%|█████████▉| 9981/10000 [44:17<00:03,  5.04it/s]

Iter 9980. Loss [[0.28098264 0.03614525 0.34972283]]


100%|█████████▉| 9991/10000 [44:20<00:02,  3.63it/s]

Iter 9990. Loss [[0.28141382 0.0369883  0.28221577]]


100%|██████████| 10000/10000 [44:22<00:00,  3.76it/s]


In [ ]:
density_graph_points, I_pdf, cdf_xy = get_set(D, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))
np.mean(yhat)

-0.36303195

In [ ]:
D_val = np.array([data_loader.validation_x, data_loader.validation_y])[:, :, 0]
density_graph_points, I_pdf, cdf_xy = get_set(D_val, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))

L_d(losses_eval, best_params, best_cop_state), np.mean(yhat)

(Array([[ 0.28595927,  0.0851197 , -0.75488394]], dtype=float32), -0.35171744)

In [ ]:
res = bootstrap(yhat, np.mean)
res.standard_error, res.confidence_interval

(0.07411516,
 ConfidenceInterval(low=-0.4968515191758786, high=-0.20814649365903487))

In [ ]:
with open('drive/MyDrive/copulae/models/{}_intcmsft.pkl'.format(label), 'wb') as fp:
    pickle.dump(best_params, fp)

# goog-fb

In [ ]:
model = PositiveBiNormalCopula(
    NormalPDFNet(layer_widths)
)

In [ ]:
data_loader = GOOG_FB()
D = np.array([data_loader.train_x, data_loader.train_y])[:, :, 0]
TrainingTensors = generate_copula_net_input(
    D=D,
    bootstrap=False
)

In [ ]:
nn_C, nn_dC, nn_c, cop_state, forward, grad = setup_training(
    model, TrainingTensors, losses
)

In [ ]:
_, subkey = jax.random.split(key)
init_params = model.init(subkey, TrainingTensors.UV_batches[0])
del subkey

params = init_params
optimizer = optax.adam(lr)
opt_state = optimizer.init(params)

In [ ]:
best = 1e6
for i in tqdm(range(n_iter)):
    grads, cop_state = grad(params, cop_state)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        loss = L_d(losses_eval, params, cop_state)
        if loss[0][-1] < best:
          best_params = params
          best_cop_state = cop_state
          best = loss[0][-1]
        print('Iter {}. Loss {}'.format(i, loss))

  0%|          | 1/10000 [01:09<192:55:46, 69.46s/it]

Iter 0. Loss [[ 0.08624099  0.2845259  18.01158   ]]


  0%|          | 12/10000 [02:07<3:00:00,  1.08s/it]

Iter 10. Loss [[0.08678316 0.27318627 7.063106  ]]


  0%|          | 21/10000 [02:09<57:01,  2.92it/s]

...

100%|█████████▉| 9982/10000 [44:22<00:03,  5.08it/s]

Iter 9980. Loss [[ 0.09147733  0.31485772 19.83263   ]]


100%|█████████▉| 9991/10000 [44:23<00:01,  5.03it/s]

Iter 9990. Loss [[ 0.0911233   0.31485584 19.83263   ]]


100%|██████████| 10000/10000 [44:26<00:00,  3.75it/s]


In [ ]:
density_graph_points, I_pdf, cdf_xy = get_set(D, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))
np.mean(yhat)

-1.0371261

In [ ]:
D_val = np.array([data_loader.validation_x, data_loader.validation_y])[:, :, 0]
density_graph_points, I_pdf, cdf_xy = get_set(D_val, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))

L_d(losses_eval, best_params, best_cop_state), np.mean(yhat)

(Array([[ 0.2574013,  0.1293621, -1.7029388]], dtype=float32), -0.913283)

In [ ]:
res = bootstrap(yhat, np.mean)
res.standard_error, res.confidence_interval

(0.089487985,
 ConfidenceInterval(low=-1.070218206939873, high=-0.7171180174567272))

In [ ]:
with open('drive/MyDrive/copulae/models/{}_googfb.pkl'.format(label), 'wb') as fp:
    pickle.dump(best_params, fp)